In [1]:
import slicer
import vtk
import numpy as np
import os
import math
import ScreenCapture
#from matplotlib import pyplot as plt

In [2]:
mrb_dir = "C:\\Users\\Edward\\Desktop\\UnionML\\MRBs"
save_dir = "C:\\Users\\Edward\\Desktop\\UnionML\\NumpyFiles05052"
screenshot_dir = "C:\\Users\\Edward\\Desktop\\UnionML\\Screenshots05052"

In [3]:
def rotation_matrix(direction, angle, point=None):
    """Return matrix to rotate about axis defined by point and direction.
    """
    sina = math.sin(angle)
    cosa = math.cos(angle)
    direction = unit_vector(direction[:3])
    # rotation matrix around unit vector
    R = np.diag([cosa, cosa, cosa])
    R += np.outer(direction, direction) * (1.0 - cosa)
    direction *= sina
    R += np.array([[0.0, -direction[2], direction[1]],
                   [direction[2], 0.0, -direction[0]],
                   [-direction[1], direction[0], 0.0]])
    M = np.identity(4)
    M[:3, :3] = R
    if point is not None:
        # rotation not around origin
        point = np.array(point[:3], dtype=np.float64, copy=False)
        M[:3, 3] = point - np.dot(R, point)
    return M

def unit_vector(data, axis=None, out=None):
    """Return ndarray normalized by length, i.e. Euclidean norm, along axis.
    """
    if out is None:
        data = np.array(data, dtype=np.float64, copy=True)
        if data.ndim == 1:
            data /= math.sqrt(np.dot(data, data))
            return data
    else:
        if out is not data:
            out[:] = np.array(data, copy=False)
        data = out
    length = np.atleast_1d(np.sum(data * data, axis))
    np.sqrt(length, length)
    if axis is not None:
        length = np.expand_dims(length, axis)
    data /= length
    if out is None:
        return data

def getSliceScreenshot(save_dir, patient):
    slices = ["Red", "Yellow", "Green"]
    for s in slices:
        sliceViewName = s
        
        # Set view background to white
        view = slicer.app.layoutManager().sliceWidget(sliceViewName).sliceView()
        view.setBackgroundColor(qt.QColor.fromRgbF(1,1,1))
        view.forceRender()

        # Capture a screenshot
        cap = ScreenCapture.ScreenCaptureLogic()
        cap.captureImageFromView(view, os.path.join(save_dir, patient + "_" + s +".png"))
        
def resizeVolume(newSpacing, originalNode, newNode):
    parameters = {}
    parameters["outputPixelSpacing"] = newSpacing
    parameters["interpolationType"] = "linear"

    ##d = slicer.cli.createNode(slicer.modules.resamplescalarvolume, parameters=None)
    ##d.GetParameterName(0,1)
    parameters["InputVolume"] = originalNode
    parameters["OutputVolume"] = newNode

    # Execute
    resampleVolume = slicer.modules.resamplescalarvolume
    cliNode = slicer.cli.runSync(resampleVolume, None, parameters)

    if cliNode.GetStatus() & cliNode.ErrorsMask:
        # error
        errorText = cliNode.GetErrorText()
        slicer.mrmlScene.RemoveNode(cliNode)
        raise ValueError("CLI execution failed: " + errorText)
        
    return newNode
    
def createCube(min_x, max_x, min_y, max_y, min_z, max_z):
    cubeSource = vtk.vtkCubeSource()
    cubeSource.SetBounds(min_z, max_z, min_y, max_y, min_x, max_x)
    cubeSource.Update()
    modelsLogic = slicer.modules.models.logic()
    cube = cubeSource.GetOutput()
    model = modelsLogic.AddModel(cube)
    model.GetDisplayNode().SetSliceIntersectionVisibility(True)
    model.GetDisplayNode().SetSliceIntersectionThickness(3)
    model.GetDisplayNode().SetColor(1,1,0)
    return model

In [4]:
def runBRAINSResample(originalVolumeNode, outputNode, transform):
    parameters = {'inputVolume':originalVolumeNode,  'outputVolume':outputNode, 'interpolationMode':'Linear', 
                  'warpTransform': transform, 'defaultValue':0, }
    slicer.cli.run(slicer.modules.brainsresample, None, parameters, wait_for_completion=True)
        
    return outputNode

In [5]:
def createVolumes2(patient):
    slicer.mrmlScene.Clear(0)
    
    print(patient)
    slicer.util.loadScene(os.path.join(mrb_dir, patient))
      
    scalar_volumes = slicer.util.getNodesByClass("vtkMRMLScalarVolumeNode")
    nodes_to_remove = []
    for i in range(len(scalar_volumes)):
        node  = slicer.util.getNodesByClass("vtkMRMLScalarVolumeNode")[i]
        if "Chest" in node.GetName() or "chest" in node.GetName():
            nodes_to_remove.append(node)
    
    for n in nodes_to_remove:
        slicer.mrmlScene.RemoveNode(n)
     
    pointListNode = slicer.mrmlScene.GetFirstNodeByClass("vtkMRMLMarkupsFiducialNode")
    originalVolumeNode = slicer.mrmlScene.GetFirstNodeByClass("vtkMRMLScalarVolumeNode")
    segmentationNode = slicer.mrmlScene.GetFirstNodeByClass("vtkMRMLSegmentationNode")
    
    print(originalVolumeNode.GetName())
    
    num_interfaces = int(pointListNode.GetNumberOfFiducials()/4)
    point_Ras = [0, 0, 0, 1]
    radius = 3

    target_x = np.asarray([1, 0, 0])
    target_y = np.asarray([0, 1, 0])
    target_z = np.asarray([0, 0, 1])
    
    placeholder = [0, 0, 0, 1]

    identityTransform = slicer.vtkMRMLTransformNode()
    tempTransform = slicer.vtkMRMLTransformNode()
    
    slicer.mrmlScene.AddNode(identityTransform)
    slicer.mrmlScene.AddNode(tempTransform)

    print("num_interfaces", num_interfaces)
    forward_transformNodes = []
    backwards_transformNodes = []
    cube_transforms = []
    extra_transforms_forward = []
    extra_transforms_backward = []
    cubes = []
    
    for i in range(num_interfaces):
        pointsRAS = []
        
        #Get new points
        for j in range(4):
            pointListNode.GetNthFiducialWorldCoordinates(4 * i + j, point_Ras)
            pointsRAS.append(point_Ras[:-1])

        pointsRASNumpy = np.transpose(np.asarray(pointsRAS))

        svd = np.linalg.svd(pointsRASNumpy - np.mean(pointsRASNumpy, axis=1, keepdims=True))
        left = svd[0]
        normal = left[:, -1]
        
        centroid = np.mean(pointsRASNumpy, axis=1, keepdims=False)
        
        target_x = np.asarray([1, 0, 0])
        target_y = np.asarray([0, 1, 0])
        target_z = np.asarray([0, 0, 1])

        angle_x = np.arccos(np.dot(target_x, normal)) * 180/np.pi
        angle_y = np.arccos(np.dot(target_y, normal)) * 180/np.pi
        angle_z = np.arccos(np.dot(target_z, normal)) * 180/np.pi
        
        chosen_dir = ""
        
        extension_x = 0
        extension_y = 0
        extension_z = 0
        
        new_rot = rotation_matrix(np.cross(normal, target_z), np.arccos(np.dot(target_z, normal)), centroid)
        
        #Rotate fiducials and CT
        forward_transformNodes.append(slicer.vtkMRMLTransformNode())
        forward_transformNodes[i].SetMatrixTransformToParent((slicer.util.vtkMatrixFromArray(new_rot)))
        slicer.mrmlScene.AddNode(forward_transformNodes[i])

        backwards_transformNodes.append(slicer.vtkMRMLTransformNode())
        backwards_transformNodes[i].SetMatrixTransformToParent((slicer.util.vtkMatrixFromArray(np.linalg.inv(new_rot))))
        slicer.mrmlScene.AddNode(backwards_transformNodes[i])
            
        pointListNode.SetAndObserveTransformNodeID(forward_transformNodes[i].GetID())
        segmentationNode.SetAndObserveTransformNodeID(forward_transformNodes[i].GetID())
        pointListNode.HardenTransform()
        segmentationNode.HardenTransform()
      
        extension_x = 20
        extension_y = 20
        extension_z = 1

        #originalVolumeNode
        transformedAndResampledVolumeNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLScalarVolumeNode")
        transformedAndResampledVolumeNode = resizeVolume([0.5, 0.5, 2], originalVolumeNode, transformedAndResampledVolumeNode)
        transformedAndResampledVolumeNode = runBRAINSResample(transformedAndResampledVolumeNode, transformedAndResampledVolumeNode, forward_transformNodes[i])
        
        best_rot = np.identity(4)
        
        extra_transforms_forward.append(slicer.vtkMRMLTransformNode())
        extra_transforms_forward[i].SetMatrixTransformToParent((slicer.util.vtkMatrixFromArray(best_rot)))
        slicer.mrmlScene.AddNode(extra_transforms_forward[i])
                                                                
        extra_transforms_backward.append(slicer.vtkMRMLTransformNode())
        extra_transforms_backward[i].SetMatrixTransformToParent((slicer.util.vtkMatrixFromArray(np.linalg.inv(best_rot))))
        slicer.mrmlScene.AddNode(extra_transforms_backward[i])                                                   
          
        #referenceVolumeNode.SetAndObserveTransformNodeID(extra_transforms_forward[i].GetID())
        
        volumeRasToIjk = vtk.vtkMatrix4x4()
        transformedAndResampledVolumeNode.GetRASToIJKMatrix(volumeRasToIjk)
        volumeIjKToRAS = vtk.vtkMatrix4x4()
        transformedAndResampledVolumeNode.GetIJKToRASMatrix(volumeIjKToRAS)
        
        centroid_ijk = [0, 0, 0, 1]
        volumeRasToIjk.MultiplyPoint(np.append(centroid, 1), centroid_ijk)
        centroid_ijk = [ int(round(c)) for c in centroid_ijk[0:3] ]
        #print(centroid_ijk)

        max_x, max_y, max_z = centroid_ijk[0] + extension_x, centroid_ijk[1] + extension_y, centroid_ijk[2] + extension_z
        min_x, min_y, min_z = centroid_ijk[0] - extension_x, centroid_ijk[1] - extension_y, centroid_ijk[2] - extension_z
        voxels = slicer.util.arrayFromVolume(transformedAndResampledVolumeNode)
        
        new_volume = voxels[min_z:max_z, min_y:max_y, min_x:max_x]
        
        file_name = patient[:-4] + "_" + str(i) + ".npy"
        
        np.save(os.path.join(save_dir, file_name), new_volume)

        #Visualization cube
        cubes.append(createCube(min_z, max_z, min_y, max_y, min_x, max_x))
        cube_transforms.append(slicer.vtkMRMLTransformNode())
        cube_transforms[i].SetMatrixTransformToParent(volumeIjKToRAS)
        slicer.mrmlScene.AddNode(cube_transforms[i])
        cubes[i].SetAndObserveTransformNodeID(cube_transforms[i].GetID())
        cubes[i].HardenTransform()
        cubes[i].SetAndObserveTransformNodeID(extra_transforms_backward[i].GetID())
        cubes[i].HardenTransform()

        #Screenshot of all slices
        
        segmentationNode = slicer.mrmlScene.GetFirstNodeByClass("vtkMRMLSegmentationNode")
        segmentationNode.GetDisplayNode().SetVisibility(0)
        
        red = slicer.mrmlScene.GetNodeByID("vtkMRMLSliceNodeRed")
        yellow = slicer.mrmlScene.GetNodeByID("vtkMRMLSliceNodeYellow")
        green = slicer.mrmlScene.GetNodeByID("vtkMRMLSliceNodeGreen")
        
        red.JumpSliceByCentering(centroid[0], centroid[1], centroid[2])
        yellow.JumpSliceByCentering(centroid[0], centroid[1], centroid[2])
        green.JumpSliceByCentering(centroid[0], centroid[1], centroid[2])
        
        red.SetFieldOfView(40,40,40)
        yellow.SetFieldOfView(40,40,40)
        green.SetFieldOfView(40,40,40)
        
        getSliceScreenshot(screenshot_dir, patient[:-4] + "_" +  str(i))
        #getScreenshot(os.path.join(screenshot_dir, patient[:-4] + "_" +  str(i) +  ".png"))
        #break

        #Rotating back
        cubes[i].SetAndObserveTransformNodeID(backwards_transformNodes[i].GetID())
        #referenceVolumeNode.SetAndObserveTransformNodeID(backwards_transformNodes[i].GetID())
        pointListNode.SetAndObserveTransformNodeID(backwards_transformNodes[i].GetID())
        segmentationNode.SetAndObserveTransformNodeID(backwards_transformNodes[i].GetID())

        cubes[i].HardenTransform()
        #referenceVolumeNode.HardenTransform()
        pointListNode.HardenTransform()
        segmentationNode.HardenTransform()
        
        #Remove node, and redo
        slicer.mrmlScene.RemoveNode(transformedAndResampledVolumeNode)
    
#     segmentationNode = slicer.mrmlScene.GetFirstNodeByClass("vtkMRMLSegmentationNode")
#     segmentationNode.GetDisplayNode().SetVisibility(1)

In [6]:
#008
#167
#265
#027*


a = createVolumes2("200.mrb")

200.mrb
3: CT 2.5 HN
num_interfaces 2
Generating customization file
Feature extraction started
Initializing output table
Starting RadiomicsCLI for Segmentation_1_segment_Model
..............Done
[2022-09-03 16:55:06] I: radiomics.script: Starting PyRadiomics (version: v3.0.1.post3+g0c53d1d)
[2022-09-03 16:55:06] I: radiomics.script: Processing input...
[2022-09-03 16:55:06] I: radiomics.featureextractor: Loading parameter file C:/Users/Edward/AppData/Local/Temp/Slicer\RadiomicsLogicParams.json
[2022-09-03 16:55:06] I: radiomics.featureextractor: Applying custom setting overrides: {'label': 1, 'correctMask': True}
[2022-09-03 16:55:06] I: radiomics.script: Input valid, starting sequential extraction from 1 case(s)...
[2022-09-03 16:55:06] I: radiomics.script: Processing case 1
[2022-09-03 16:55:06] I: radiomics.featureextractor: Calculating features with label: 1
[2022-09-03 16:55:06] I: radiomics.featureextractor: Loading image and mask
[2022-09-03 16:55:07] W: radiomics.imageoperation


Starting RadiomicsCLI for Segmentation_1_segment_Model_1
.......................................................Done
[2022-09-03 16:57:02] I: radiomics.script: Starting PyRadiomics (version: v3.0.1.post3+g0c53d1d)
[2022-09-03 16:57:02] I: radiomics.script: Processing input...
[2022-09-03 16:57:02] I: radiomics.featureextractor: Loading parameter file C:/Users/Edward/AppData/Local/Temp/Slicer\RadiomicsLogicParams.json
[2022-09-03 16:57:02] I: radiomics.featureextractor: Applying custom setting overrides: {'label': 1, 'correctMask': True}
[2022-09-03 16:57:02] I: radiomics.script: Input valid, starting sequential extraction from 1 case(s)...
[2022-09-03 16:57:02] I: radiomics.script: Processing case 1
[2022-09-03 16:57:02] I: radiomics.featureextractor: Calculating features with label: 1
[2022-09-03 16:57:02] I: radiomics.featureextractor: Loading image and mask
[2022-09-03 16:57:03] W: radiomics.imageoperations: Image/Mask geometry mismatch, attempting to correct Mask
[2022-09-03 16:57

In [15]:
def getSliceThickness(patient):
    slicer.mrmlScene.Clear(0)
    
    #print(patient)
    slicer.util.loadScene(os.path.join(mrb_dir, patient))
      
    scalar_volumes = slicer.util.getNodesByClass("vtkMRMLScalarVolumeNode")
    nodes_to_remove = []
    for i in range(len(scalar_volumes)):
        node  = slicer.util.getNodesByClass("vtkMRMLScalarVolumeNode")[i]
        if "Chest" in node.GetName() or "chest" in node.GetName():
            nodes_to_remove.append(node)
    
    for n in nodes_to_remove:
        slicer.mrmlScene.RemoveNode(n)
     
    pointListNode = slicer.mrmlScene.GetFirstNodeByClass("vtkMRMLMarkupsFiducialNode")
    originalVolumeNode = slicer.mrmlScene.GetFirstNodeByClass("vtkMRMLScalarVolumeNode")

    print(patient, originalVolumeNode.GetSpacing())

In [5]:
def getNumberOfVolumeNodes(patient):
    slicer.mrmlScene.Clear(0)
    slicer.util.loadScene(os.path.join(mrb_dir, patient))
      
    scalar_volumes = slicer.util.getNodesByClass("vtkMRMLScalarVolumeNode")
    print(patient, len(scalar_volumes))

In [6]:
all_patients = os.listdir(mrb_dir)
for p in all_patients:
    if int(p[:-4]) > 222:
    #if int(p[:-4]) == 200:
        #createVolumes2(p)
    #getSliceThickness(p)
        getNumberOfVolumeNodes(p)

224.mrb 1
227.mrb 1
228.mrb 3
233.mrb 1
236.mrb 1
237.mrb 1
242.mrb 1
248.mrb 1
259.mrb 2
260.mrb 1
262.mrb 1
263.mrb 1
265.mrb 3
267.mrb 3
268.mrb 1
274.mrb 1
278.mrb 1
280.mrb 2
282.mrb 1
286.mrb 1
317.mrb 1
325.mrb 1
326.mrb 1
330.mrb 1
338.mrb 5
343.mrb 1
373.mrb 3
391.mrb 4
403.mrb 1
407.mrb 1
434.mrb 1
436.mrb 1
443.mrb 2
456.mrb 2
471.mrb 1
476.mrb 2
486.mrb 1
491.mrb 1
500.mrb 1
505.mrb 1
512.mrb 1
524.mrb 2
527.mrb 1
528.mrb 1
530.mrb 1
538.mrb 1
544.mrb 3
545.mrb 1
548.mrb 1
550.mrb 1
575.mrb 1
576.mrb 1
577.mrb 1
578.mrb 1


In [10]:
#redo

#to_redo = ["015", "062", "100", "456", "491"]
to_redo = ["191", "265"]

for p in to_redo:
    createVolumes(p + ".mrb")


191.mrb
3: CT 2.5 HN
num_interfaces 3
voxel sum 45318731
voxel sum 35796038
voxel sum 40512503
265.mrb
606: NECK COR STD 3
num_interfaces 3
voxel sum 20765718
voxel sum 18098674
voxel sum 25419918


In [159]:
originalVolumeNode.GetName()

'601: 2 5MM AXIALS - imageOrientationPatient 2'